In [257]:
#!pip install missingno
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import missingno as msno
from sklearn.tree import DecisionTreeClassifier

In [275]:
train = pd.read_csv("train_fix_09_12_3.csv")
test = pd.read_csv("test_fix_09_12_3.csv")

In [276]:
y_train= train['Reached.on.Time_Y.N']
X_train = train.drop('Reached.on.Time_Y.N', axis=1)

In [277]:
X_train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6999 entries, 0 to 6998
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Cost_of_the_Product     6999 non-null   int64  
 1   Discount_offered        6999 non-null   float64
 2   Weight_in_gms           6999 non-null   int64  
 3   Product_importance_low  6999 non-null   int64  
dtypes: float64(1), int64(3)
memory usage: 218.8 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Cost_of_the_Product     4000 non-null   int64  
 1   Discount_offered        4000 non-null   float64
 2   Weight_in_gms           4000 non-null   int64  
 3   Product_importance_low  4000 non-null   int64  
dtypes: float64(1), int64(3)
memory usage: 125.1 KB


In [278]:
train.describe()
test.describe()

,Cost_of_the_Product,Discount_offered,Weight_in_gms,Product_importance_low
count,4000.000000,4000.000000,4000.000000,4000.000000
mean,210.420250,11.710750,3597.761000,0.468250
std,47.755542,11.548222,1593.715242,0.499053
min,96.000000,1.000000,1003.000000,0.000000
25%,170.000000,7.000000,1876.000000,0.000000
50%,214.000000,10.000000,4045.500000,0.000000
75%,251.000000,10.000000,4987.500000,1.000000
max,310.000000,65.000000,7684.000000,1.000000


# XGB

In [279]:
# !pip install xgboost

In [280]:
# XGBoost 라이브러리 불러오기
from xgboost import XGBClassifier

# 모델 선언 및 하이퍼파라미터 설정
xgb_model = XGBClassifier()  # 하이퍼파라미터 설정 없이 기본값으로 모델을 생성합니다.

# 학습
xgb_model.fit(X_train, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [281]:
param = {
    'max_depth': [1],
    'n_estimators':[16]
}

xgb_model = GradientBoostingClassifier(random_state=10)

from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(xgb_model, param, cv=5, scoring='accuracy', n_jobs=-1, verbose=3)
grid.fit(X_train, y_train)
print(grid.best_params_)
print(grid.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
{'max_depth': 1, 'n_estimators': 16}
0.6685262942918412


# AdaBoost

In [282]:
# 모델 불러오기
from sklearn.ensemble import AdaBoostClassifier

In [283]:
# 모델 생성, 하이퍼 파라미터 설정
ada_model = AdaBoostClassifier() 

# 학습
ada_model.fit(X_train, y_train)

AdaBoostClassifier()

In [284]:
# 교차검증으로 평가
from sklearn.model_selection import cross_val_score
#sco = cross_val_score(forest_model, X_train, y_train, cv=5)
cross_val_score(ada_model, X_train, y_train, cv=5).mean()



0.6539548657204125

In [285]:
param = {
}

gbm_model2 = GradientBoostingClassifier(random_state=1)

from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(ada_model, param, cv=5, scoring='accuracy', n_jobs=-1, verbose=2)
grid.fit(X_train, y_train)
print(grid.best_params_)
print(grid.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
{}
0.6539548657204125


# GBM

In [286]:
# 모델 라이브러리 불러오기
from sklearn.ensemble import GradientBoostingClassifier

# 모델 생성, 하이퍼파라미터 설정
gbm_model = GradientBoostingClassifier()

# 학습
gbm_model.fit(X_train, y_train)

GradientBoostingClassifier()

In [287]:
# 교차검증으로 평가
from sklearn.model_selection import cross_val_score
#sco = cross_val_score(forest_model, X_train, y_train, cv=5)
cross_val_score(gbm_model, X_train, y_train, cv=12).mean()



0.6665259306687971

In [297]:
param = {
    'max_depth': [3,4,5],
    'min_samples_split': [9,10],
    'n_estimators':[19,20,21],
}

gbm_model = GradientBoostingClassifier(random_state=110)

from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(gbm_model, param, cv=5, scoring='accuracy', n_jobs=-1, verbose=3)
grid.fit(X_train, y_train)
print(grid.best_params_)
print(grid.best_score_)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
{'max_depth': 4, 'min_samples_split': 9, 'n_estimators': 20}
0.6759570101092617


In [298]:
pre = grid.predict(test)

In [300]:
sub = pd.read_csv('./sampleSubmission.csv')
sub['Reached.on.Time_Y.N']=pre
sub.to_csv('09_12_gbm_3.csv', index=False)

# LGBM

In [291]:
!pip install lightgbm

In [292]:
import lightgbm
from lightgbm.sklearn import LGBMClassifier

# 모델 생성, 하이퍼파라미터 설정 X
lgbm_model = LGBMClassifier()

# 학습
lgbm_model.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 4162, number of negative: 2837
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000098 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 535
[LightGBM] [Info] Number of data points in the train set: 6999, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.594656 -> initscore=0.383249
[LightGBM] [Info] Start training from score 0.383249


LGBMClassifier()

In [293]:
# 교차검증으로 평가
from sklearn.model_selection import cross_val_score
#sco = cross_val_score(forest_model, X_train, y_train, cv=5)
cross_val_score(lgbm_model, X_train, y_train, cv=5).mean()

[LightGBM] [Info] Number of positive: 3329, number of negative: 2270
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000047 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 532
[LightGBM] [Info] Number of data points in the train set: 5599, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.594570 -> initscore=0.382892
[LightGBM] [Info] Start training from score 0.382892
[LightGBM] [Info] Number of positive: 3329, number of negative: 2270
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000038 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 529
[LightGBM] [Info] Number of data points in the train set: 5599, number of used features: 4
[LightGBM] [Info] [binary:Boos

0.6566695598897171

In [294]:
param = {
    'max_depth': [1],
    'min_samples_split': [1],
    'n_estimators':[17],
}

lgbm_model = LGBMClassifier(random_state=0)

from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(lgbm_model, param, cv=5, scoring='accuracy', n_jobs=-1, verbose=3)
grid.fit(X_train, y_train)
print(grid.best_params_)
print(grid.best_score_)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[LightGBM] [Warning] Unknown parameter: min_samples_split
[LightGBM] [Warning] Unknown parameter: min_samples_split
[LightGBM] [Info] Number of positive: 4162, number of negative: 2837
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000149 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 535
[LightGBM] [Info] Number of data points in the train set: 6999, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.594656 -> initscore=0.383249
[LightGBM] [Info] Start training from score 0.383249
{'max_depth': 1, 'min_samples_split': 1, 'n_estimators': 17}
0.6685261921780864
